In [1]:
import numpy as np
import torch
from torch import nn, optim, autograd
from torch.nn import functional as F
from pyDOE import lhs
import scipy.io
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.gridspec as gridspec
%matplotlib inline
from mpl_toolkits.axes_grid1 import make_axes_locatable

from models_all import *

#Paper reproduction
torch.manual_seed(1234)
torch.cuda.manual_seed(1234)
np.random.seed(1234)

# Data For PINNs

In [2]:
N_train = 5000
N_bound = 100

np.random.seed(123)
train_x_y_t = lhs(3,N_train)
np.random.seed(223)
bound_x_y = lhs(2,N_bound)
bound_x_y_t = np.concatenate((bound_x_y,np.zeros((N_bound,1))),1)

In [3]:
train_x_y_t = torch.from_numpy(train_x_y_t).float()
bound_x_y = torch.from_numpy(bound_x_y).float()
bound_x_y_t = torch.from_numpy(bound_x_y_t).float()

x = train_x_y_t[:,0:1]
y = train_x_y_t[:,1:2]
t = train_x_y_t[:,2:3]
x.requires_grad_()
y.requires_grad_()
t.requires_grad_()

tensor([[0.3976],
        [0.7691],
        [0.9756],
        ...,
        [0.2023],
        [0.6423],
        [0.6873]], requires_grad=True)

## Get Data

In [4]:
def relative_l2(u_pred,u_real):
    l2 = np.linalg.norm(u_real-u_pred,2)/np.linalg.norm(u_real,2)
    return l2

In [5]:
import scipy.integrate
from numpy import exp
from math import sqrt
import math
def InIn(x_y_t):
    x1 = x_y_t[:,0:1]
    x2 = x_y_t[:,1:2]
    t = x_y_t[:,2:3]
    P = np.zeros_like(t)
    for i in range(len(t)):
        f = lambda y1,y2 : exp(-((x1[i]-y1)**2+(x2[i]-y2)**2)/(4*t[i]))*(y1-y2)
        p,err= scipy.integrate.dblquad(f, -np.inf, np.inf, lambda g : -np.inf, lambda h : np.inf)
        P[i] = p/(4*np.pi*t[i])
    return P

In [6]:
np.random.seed(5678)
test_x_y_t = lhs(3,500)
x1 = test_x_y_t[:,0:1]
x2 = test_x_y_t[:,1:2]
t_ = test_x_y_t[:,2:3]
test_u = np.zeros_like(t_)
for i in range(len(t_)):
    f = lambda y1,y2 : exp(-((x1[i]-y1)**2+(x2[i]-y2)**2)/(4*t_[i]))*(y1-y2)
    p,err= scipy.integrate.dblquad(f, -np.inf, np.inf, lambda g : -np.inf, lambda h : np.inf)
    test_u[i] = p/(4*np.pi*t_[i])


In [7]:
test_x_y_t = torch.from_numpy(test_x_y_t).float()

### Labeled Data

In [8]:
n = 12
np.random.seed(234)
gan_data_x_t_a = lhs(3,10)
np.random.seed(23456)
gan_data_x_t_b = lhs(3,n-10)
gan_data_x_t = np.concatenate((gan_data_x_t_a,gan_data_x_t_b),axis=0)
gan_data_u = InIn(gan_data_x_t)

In [9]:
gan_data_x_t = torch.from_numpy(gan_data_x_t).float()
gan_data_u = torch.from_numpy(gan_data_u).float()

# Method2
**$\text { GA - PINN }$**

<div class="alert alert-info">`loss_function：`

$\mathrm{L}_D=\frac{1}{J} \sum_{j=1}^J\left(1-D\left[\left(\mathbf{x}_T^{(j)}, u_T^{(j)}\right)\right]\right)+D\left[\left(x_L^{(j)}, G\left[x_L^{(j)}\right]\right)\right] \\
\mathrm{L}_G=\mathrm{L}_T+\frac{1}{J} \sum_{j=1}^J\left(1-D\left[\left(x_T^{(j)}, G\left[x_T^{(j)}\right]\right)\right]\right)\\
\mathrm{L}_{\text {PINN }}:=\mathrm{L}_f+\lambda_1 \mathrm{~L}_b \text { with } \mathrm{L}_b:=\sum_{i=1}^I \mathrm{~L}_{b_i}
$



</div>

In [10]:
torch.manual_seed(1234)
torch.cuda.manual_seed(1234)
np.random.seed(1234)

PINNs2 = NN_H2(3, 100, 4, 1)
PINNs2.apply(weights_init)
optimizer1 = optim.Adam([{'params': PINNs2.parameters()}], lr=1e-3)

discriminator= get_discriminator(4, 100, 1, 1)
discriminator.apply(weights_init)
optimizer2 = optim.Adam([{'params': discriminator.parameters(), 'weight_decay': 0.01}], lr=5e-3)


loss_all_2 = []
test_loss_2 = []

#########gpu############
discriminator.cuda()
gan_data_x_t = gan_data_x_t.cuda()
gan_data_u = gan_data_u.cuda()
bound_x_y_t = bound_x_y_t.cuda()
bound_x_y = bound_x_y.cuda()
PINNs2.cuda()
x = x.cuda()
y = y.cuda()
t = t.cuda()
#########gpu############


loss1_value = 1
it = 0
while  loss1_value>5e-6:
    PINNs2.cuda()
    
    ##############loss D############
    pre_H = PINNs2(gan_data_x_t)
    d_fake = discriminator(torch.cat((gan_data_x_t,pre_H.detach()),1))
    d_real = discriminator(torch.cat((gan_data_x_t,gan_data_u),1))
    
    loss_d = torch.mean(1-d_real)+torch.mean(d_fake)
    
    optimizer2.zero_grad()
    loss_d .backward()
    optimizer2.step()  
    
    ##############loss G ############
    pre_H = PINNs2(gan_data_x_t)
    d_fake = discriminator(torch.cat((gan_data_x_t,pre_H.detach()),1))
    loss_L = torch.mean(torch.square(pre_H - gan_data_u))+torch.mean(1-d_fake)
    
    optimizer1.zero_grad()
    loss_L.backward()
    optimizer1.step()  
    
    ##### loss_Bi  ######
    u_bound = PINNs2(bound_x_y_t)
    
    loss_bound = torch.mean(torch.square(u_bound-(bound_x_y[:,0:1]-bound_x_y[:,1:2])))
  
    ##### loss f  ######
    
    u_inside = PINNs2(torch.cat((x,y,t),1))
    u_x = autograd.grad(outputs=u_inside, inputs=x,
                              grad_outputs=torch.ones_like(u_inside),
                              create_graph=True, retain_graph=True, only_inputs=True)[0]
    u_xx = autograd.grad(outputs=u_x, inputs=x,
                              grad_outputs=torch.ones_like(u_x),
                              create_graph=True, retain_graph=True, only_inputs=True)[0]
    u_y = autograd.grad(outputs=u_inside, inputs=y,
                              grad_outputs=torch.ones_like(u_inside),
                              create_graph=True, retain_graph=True, only_inputs=True)[0]
    u_yy = autograd.grad(outputs=u_y, inputs=y,
                              grad_outputs=torch.ones_like(u_y),
                              create_graph=True, retain_graph=True, only_inputs=True)[0]  
    u_t = autograd.grad(outputs=u_inside, inputs=t,
                          grad_outputs=torch.ones_like(u_inside),
                          create_graph=True, retain_graph=True, only_inputs=True)[0]
    deata_u = u_xx+u_yy
    loss_f = torch.mean(torch.square(deata_u-u_t))
    
    loss = loss_bound+loss_f
    loss1_value = loss.item()
    
    loss_all_2.append(loss1_value)
    optimizer1.zero_grad()
    loss.backward()
    optimizer1.step()
    
    #########  test_loss NRMSE  #########
    PINNs2.cpu()
    test_loss =  relative_l2(PINNs2(test_x_y_t).detach().numpy(),test_u)
    test_loss_2.append(test_loss)
    
    if it % 100 == 0:
        print('It:', it, 'Loss:', loss.item())
    it = it + 1        
loss1_value    

It: 0 Loss: 0.1490023136138916
It: 100 Loss: 0.0006457681301981211
It: 200 Loss: 0.011818177066743374
It: 300 Loss: 0.00018325836572330445
It: 400 Loss: 8.742836507735774e-05
It: 500 Loss: 0.0008287227828986943
It: 600 Loss: 9.441892325412482e-05
It: 700 Loss: 5.158056956133805e-05
It: 800 Loss: 7.102534436853603e-05
It: 900 Loss: 0.00017430355364922434
It: 1000 Loss: 4.2615636630216613e-05
It: 1100 Loss: 6.0848378780065104e-05
It: 1200 Loss: 0.0004768115468323231
It: 1300 Loss: 3.5336805012775585e-05
It: 1400 Loss: 0.00012806140875909477
It: 1500 Loss: 2.666324689926114e-05
It: 1600 Loss: 5.015214264858514e-05
It: 1700 Loss: 1.2317969776631799e-05
It: 1800 Loss: 2.2213958800421096e-05
It: 1900 Loss: 0.0001275196555070579
It: 2000 Loss: 8.48605213832343e-06


4.9606087486608885e-06

# Method3
$\text { PINN }^{\dagger}$

<div class="alert alert-info">objective function：

$\overline{\mathrm{L}}_{\text {PINN }} =\mathrm{L}_{\text {PINN }}+\lambda_2\mathrm{L}_T$



</div>

In [11]:
k = 2

In [12]:
torch.manual_seed(1234)
torch.cuda.manual_seed(1234)
np.random.seed(1234)

PINNs3 = NN_H2(3, 100, 4, 1)
PINNs3.apply(weights_init)
optimizer1 = optim.Adam([{'params': PINNs3.parameters()}], lr=1e-3)

loss_all_3 = []
test_loss_3 = []

#########gpu############
bound_x_y_t = bound_x_y_t.cuda()
bound_x_y = bound_x_y.cuda()
PINNs3.cuda()
x = x.cuda()
y = y.cuda()
t = t.cuda()
gan_data_x_t = gan_data_x_t.cuda()
gan_data_u = gan_data_u.cuda()
#########gpu############

loss1_value = 1
it = 0
while  loss1_value>5e-6:
    PINNs3.cuda()
    
    ##### loss_Bi  ######
    u_bound = PINNs3(bound_x_y_t)
    
    loss_bound = torch.mean(torch.square(u_bound-(bound_x_y[:,0:1]-bound_x_y[:,1:2])))
  
    ##### loss f  ######
    
    u_inside = PINNs3(torch.cat((x,y,t),1))
    u_x = autograd.grad(outputs=u_inside, inputs=x,
                              grad_outputs=torch.ones_like(u_inside),
                              create_graph=True, retain_graph=True, only_inputs=True)[0]
    u_xx = autograd.grad(outputs=u_x, inputs=x,
                              grad_outputs=torch.ones_like(u_x),
                              create_graph=True, retain_graph=True, only_inputs=True)[0]
    u_y = autograd.grad(outputs=u_inside, inputs=y,
                              grad_outputs=torch.ones_like(u_inside),
                              create_graph=True, retain_graph=True, only_inputs=True)[0]
    u_yy = autograd.grad(outputs=u_y, inputs=y,
                              grad_outputs=torch.ones_like(u_y),
                              create_graph=True, retain_graph=True, only_inputs=True)[0]  
    u_t = autograd.grad(outputs=u_inside, inputs=t,
                          grad_outputs=torch.ones_like(u_inside),
                          create_graph=True, retain_graph=True, only_inputs=True)[0]
    deata_u = u_xx+u_yy
    loss_f = torch.mean(torch.square(deata_u-u_t))
    
    #####loss PI#######
    loss = loss_bound+loss_f
    pre_H = PINNs3(gan_data_x_t)
    
    loss_p = k*torch.mean(torch.square(pre_H - gan_data_u))+loss
    
    
    loss1_value = loss.item()
    
    
    loss_all_3.append(loss1_value)
    optimizer1.zero_grad()
    loss_p.backward()
    optimizer1.step()
    
    #########  test_loss NRMSE  #########
    PINNs3.cpu()
    test_loss =  relative_l2(PINNs3(test_x_y_t).detach().numpy(),test_u)
    test_loss_3.append(test_loss)
    
    if it % 100 == 0:
        print('It:', it, 'Loss:', loss.item(), 'testloss:', test_loss)
    it = it + 1        
loss1_value    

It: 0 Loss: 0.3055083453655243 testloss: 0.9181224264664686
It: 100 Loss: 0.0018943955656141043 testloss: 0.07947364053507526
It: 200 Loss: 0.0006960901082493365 testloss: 0.04625939514302399
It: 300 Loss: 0.00039155076956376433 testloss: 0.034495477476634465
It: 400 Loss: 0.00021166018268559128 testloss: 0.02695049668806417
It: 500 Loss: 0.0013928492553532124 testloss: 0.030612320942439236
It: 600 Loss: 0.00012038713612128049 testloss: 0.020274782820686087
It: 700 Loss: 8.545262971892953e-05 testloss: 0.01984006154391684
It: 800 Loss: 6.565804505953565e-05 testloss: 0.019444508250467382
It: 900 Loss: 0.00014664430636912584 testloss: 0.019142009508609403
It: 1000 Loss: 8.609797805547714e-05 testloss: 0.016576007447156302
It: 1100 Loss: 6.327931623673066e-05 testloss: 0.016458438784542714
It: 1200 Loss: 4.911975338472985e-05 testloss: 0.016220122665231346
It: 1300 Loss: 3.935201311833225e-05 testloss: 0.015862206947114807
It: 1400 Loss: 9.07165594981052e-05 testloss: 0.01404632623619317

4.997066753276158e-06

# Method4 
$\text { GA - PINN }^{\dagger}$

<div class="alert alert-info">objective function：

$
\mathrm{L}_D=\frac{1}{J} \sum_{j=1}^J\left(1-D\left[\left(\mathbf{x}_T^{(j)}, u_T^{(j)}\right)\right]\right)+D\left[\left(x_L^{(j)}, G\left[x_L^{(j)}\right]\right)\right] \\
\mathrm{L}_G=\mathrm{L}_T+\frac{1}{J} \sum_{j=1}^J\left(1-D\left[\left(x_T^{(j)}, G\left[x_T^{(j)}\right]\right)\right]\right)\\
\overline{\mathrm{L}}_{\text {PINN }} =\mathrm{L}_{\text {PINN }}+\lambda_2\mathrm{L}_T
$



</div>


In [13]:
torch.manual_seed(1234)
torch.cuda.manual_seed(1234)
np.random.seed(1234)

PINNs4 = NN_H2(3, 100, 4, 1)
PINNs4.apply(weights_init)
optimizer1 = optim.Adam([{'params': PINNs4.parameters()}], lr=1e-3)

discriminator= get_discriminator(4, 100, 1, 1)
discriminator.apply(weights_init)
optimizer2 = optim.Adam([{'params': discriminator.parameters(), 'weight_decay': 0.01}], lr=5e-3)

loss_all_4 = []
test_loss_4 = []

#########gpu############
discriminator.cuda()
gan_data_x_t = gan_data_x_t.cuda()
gan_data_u = gan_data_u.cuda()
bound_x_y_t = bound_x_y_t.cuda()
bound_x_y = bound_x_y.cuda()
PINNs4.cuda()
x = x.cuda()
y = y.cuda()
t = t.cuda()
#########gpu############


loss1_value = 1
it = 0
while  loss1_value>5e-6:
    PINNs4.cuda()
    
    ##############loss D############
    pre_H = PINNs4(gan_data_x_t)
    d_fake = discriminator(torch.cat((gan_data_x_t,pre_H.detach()),1))
    d_real = discriminator(torch.cat((gan_data_x_t,gan_data_u),1))
    
    loss_d = torch.mean(1-d_real)+torch.mean(d_fake)
    
    optimizer2.zero_grad()
    loss_d .backward()
    optimizer2.step()  
    
    ##############loss G############
    pre_H = PINNs4(gan_data_x_t)
    d_fake = discriminator(torch.cat((gan_data_x_t,pre_H.detach()),1))
    loss_L = torch.mean(torch.square(pre_H - gan_data_u))+torch.mean(1-d_fake)
    
    optimizer1.zero_grad()
    loss_L.backward()
    optimizer1.step()  

    ##### loss_Bi  ######
    u_bound = PINNs4(bound_x_y_t)
    
    loss_bound = torch.mean(torch.square(u_bound-(bound_x_y[:,0:1]-bound_x_y[:,1:2])))
  
    ##### loss f  ######
    
    u_inside = PINNs4(torch.cat((x,y,t),1))
    u_x = autograd.grad(outputs=u_inside, inputs=x,
                              grad_outputs=torch.ones_like(u_inside),
                              create_graph=True, retain_graph=True, only_inputs=True)[0]
    u_xx = autograd.grad(outputs=u_x, inputs=x,
                              grad_outputs=torch.ones_like(u_x),
                              create_graph=True, retain_graph=True, only_inputs=True)[0]
    u_y = autograd.grad(outputs=u_inside, inputs=y,
                              grad_outputs=torch.ones_like(u_inside),
                              create_graph=True, retain_graph=True, only_inputs=True)[0]
    u_yy = autograd.grad(outputs=u_y, inputs=y,
                              grad_outputs=torch.ones_like(u_y),
                              create_graph=True, retain_graph=True, only_inputs=True)[0]  
    u_t = autograd.grad(outputs=u_inside, inputs=t,
                          grad_outputs=torch.ones_like(u_inside),
                          create_graph=True, retain_graph=True, only_inputs=True)[0]
    deata_u = u_xx+u_yy
    loss_f = torch.mean(torch.square(deata_u-u_t))
    
    #####loss PI#######
    loss = loss_bound+loss_f
    pre_H = PINNs4(gan_data_x_t)
    
    loss_p = k*torch.mean(torch.square(pre_H - gan_data_u))+loss
    
    
    loss1_value = loss.item()
    
    
    loss_all_4.append(loss1_value)
    optimizer1.zero_grad()
    loss_p.backward()
    optimizer1.step()
    
    #########  test_loss NRMSE  #########
    PINNs4.cpu()
    test_loss =  relative_l2(PINNs4(test_x_y_t).detach().numpy(),test_u)
    test_loss_4.append(test_loss)
    
    if it % 100 == 0:
        print('It:', it, 'Loss:', loss.item(), 'testloss:', test_loss)
    it = it + 1        
loss1_value    

It: 0 Loss: 0.1490023136138916 testloss: 0.6239985424560816
It: 100 Loss: 0.0010363127803429961 testloss: 0.03525611741294073
It: 200 Loss: 0.0004527221026364714 testloss: 0.0251721060193091
It: 300 Loss: 0.0005369295249693096 testloss: 0.017521290303356356
It: 400 Loss: 0.00022278081451077014 testloss: 0.014901772870093371
It: 500 Loss: 0.000126101280329749 testloss: 0.014300541808405077
It: 600 Loss: 0.0002851778408512473 testloss: 0.012779372575165433
It: 700 Loss: 0.00013495172606781125 testloss: 0.011196853576236196
It: 800 Loss: 7.878628093749285e-05 testloss: 0.010735646563975444
It: 900 Loss: 0.0001617727248230949 testloss: 0.009611630513104775
It: 1000 Loss: 7.991266465978697e-05 testloss: 0.00929146162275445
It: 1100 Loss: 0.00012140625767642632 testloss: 0.008264023326750433
It: 1200 Loss: 5.619384683086537e-05 testloss: 0.008502676879787832
It: 1300 Loss: 0.0001002189310383983 testloss: 0.007892151649264177
It: 1400 Loss: 2.6621906727086753e-05 testloss: 0.00787504918944469

4.960638761986047e-06

In [14]:
test_loss_2 = np.array(test_loss_2)
test_loss_3 = np.array(test_loss_3)
test_loss_4 = np.array(test_loss_4)

In [16]:
np.save('../../experimental_data/J_12/test_loss_2',test_loss_2)
np.save('../../experimental_data/J_12/test_loss_3',test_loss_3)
np.save('../../experimental_data/J_12/test_loss_4',test_loss_4)